# 基于Mindspore构造负对数似然损失函数

 本小节主要介绍构造负对数似然损失函数的设计，使用Negative Log-Likelihood Loss损失函数作为讲解实例。

# 1. 实验目的
- 理解Negative Log-Likelihood Loss损失函数的意义
- 使用MindSpore框架实现的Negative Log-Likelihood Loss

# 2. Negative Log-Likelihood Loss损失函数知识点介绍

负对数似然损失函数（Negative Log-Likelihood Loss）是在概率建模和分类任务中常用的损失函数之一。它通常用于多类别分类问题，其中每个样本被分为多个互斥的类别。

## 2.1 基础知识
- 交叉熵函数和负对数似然损失函数在某种程度上是等价的。在分类任务中，交叉熵函数通常用于衡量模型输出和真实标签之间的差异，而负对数似然损失函数则用于最大化模型对观测数据的似然。
- 对于二分类问题，交叉熵函数和负对数似然损失函数是完全等价的。对于多类别分类问题，两者在形式上稍有不同，但是在数学上是等价的。具体而言，对于多类别分类问题，交叉熵函数是对所有类别的负对数似然损失函数的平均值。
- 需要注意的是，使用交叉熵函数计算损失时，通常要结合softmax或sigmoid等激活函数的输出，以将输出值转化为概率分布。而负对数似然损失函数通常用于直接计算概率分布的对数似然。
- 总的来说，交叉熵函数和负对数似然损失函数在分类任务中是等价的，可以根据实际需求选择使用其中之一。


## 2.2 似然函数

假设X 是观测结果序列，它的概率分布$f_x$依赖于参数$\theta$ ，则似然函数表示为


$$L(\theta|x) = f_\theta(x) =P_\theta(X=x)$$

$L(\theta|x)$为参数$\theta$的似然函数,x为随机变量X的输出。


## 2.3 负对数似然损失函数
负对数似然损失的数学定义如下：

$$L(x, y) = \frac {-1}{N} * ∑(∑(y_i) * log(x_i))$$

其中，N表示样本数量，∑表示对所有样本和类别的求和操作。

负对数似然损失是将真实标签和模型预测的概率分布进行逐元素相乘，然后取对数，并求和得到损失值。
损失越小，模型的预测概率分布与真实标签越接近。

NLLLoss函数虽然叫负对数似然损失函数，但是该函数内部并没有像公式里那样进行了对数计算，而是在激活函数上使用了nn.LogSoftmax()函数。
所以NLLLoss函数只是做了求和取平均然后再取反的计算，在使用时要配合logsoftmax函数一起使用，或者直接使用交叉熵损失函数。


# 3. 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4. 数据处理

## 4.1 数据准备

   在本次实验中，我们使用numpy随机生成的样本数据集进行测试。

## 4.2 数据处理

使用numpy生成测试数据：

$$[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]$$

创建2个样本在3个类别的预测概率：

$$[[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]] $$


创建它们对应的标签：
$$[0, 2] $$

In [1]:
#导入mindspore框架。
import mindspore
#将Model用于创建模型对象，完成网络搭建和编译，并用于训练和评估
from mindspore import nn 
#导入可用于Cell的构造函数的算子。
from mindspore import ops 

In [2]:
# 给定一个矩阵X，我们可以对所有元素求和。
X = mindspore.Tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], mindspore.float32)
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

(Tensor(shape=[1, 3], dtype=Float32, value=
 [[ 5.00000000e+00,  7.00000000e+00,  9.00000000e+00]]),
 Tensor(shape=[2, 1], dtype=Float32, value=
 [[ 6.00000000e+00],
  [ 1.50000000e+01]]))

# 5、实验过程

模型的构建包括如下部分：
- 导入库和函数
- 自定义损失函数Negative Log-Likelihood Loss

## 5.1 导入相关库

In [4]:
import numpy as np
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
import mindspore.numpy as mnp
from mindspore import Tensor, Parameter
from mindspore.common.initializer import initializer, Zero, Normal

# 6、模型构建

损失函数NLLLoss自定义完成后，创建一个数据样本及它们对应的标签y，使用x作为x_sample中概率的索引来实现负对数似然损失函数。

In [ ]:
# softmax函数用于将其转换为概率分布。softmax函数对每个样本（行）的元素进行指数运算，
# 然后对每个样本的元素求和，得到一个分区（partition）向量，其中每个元素表示对应样本的所有类别的指数和。
def softmax(X):
    X_exp = ops.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

In [6]:
# 定义模型
class Net(nn.Cell):
    def __init__(self, num_inputs, num_outputs):
        super().__init__
        self.W = Parameter(initializer(Normal(0.01, 0), (num_inputs, num_outputs), mindspore.float32))
        self.b = Parameter(initializer(Zero(), num_outputs, mindspore.float32))

    def construct(self, X):
        return softmax(ops.matmul(X.reshape((-1, self.W.shape[0])), self.W) + self.b)

创建一个数据样本x_sample，其中包含2个样本在3个类别的预测概率，及它们对应的标签x，使用x作为x_sample中概率的索引。

In [7]:
# 生成标签
x = mindspore.Tensor([0, 2], mindspore.int32)
# 生成样本
x_sample = mindspore.Tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]], mindspore.float32)
x_sample[[0, 1], x]

Tensor(shape=[2], dtype=Float32, value= [ 1.00000001e-01,  5.00000000e-01])

In [8]:
# 实现负对数似然损失函数
def NLLLoss(x_sample, x):
    return -mnp.log(x_sample[mnp.arange(x_sample.shape[0]), x])
NLLLoss(x_sample, x)

Tensor(shape=[2], dtype=Float32, value= [ 2.30258751e+00,  6.93148613e-01])

# 7、模型预测

定义一个计算精确度的函数，输入预测值和真实值，选择NLLLoss计算模型的精确度。accuracy函数接收两个输入：x_sample表示模型的预测输出，x表示真实的目标标签。

In [9]:
# 计算预测正确的数量
def accuracy(x_sample, x): 
    # 函数检查x_sample的形状是否大于1维且第二个维度大于1。
    # 如果满足条件，说明x_sample是多类别预测的概率分布结果，需要使用argmax函数获取最大概率对应的类别索引。 
    if len(x_sample.shape) > 1 and x_sample.shape[1] > 1:  
        x_sample = x_sample.argmax(axis=1)
    # 使用float(cmp.sum())计算预测正确的数量，cmp.sum()返回布尔数组中为True的元素数量，即预测正确的数量。              
    cmp = x_sample.asnumpy() == x.asnumpy()            
    return float(cmp.sum())

# 将预测正确的数量除以样本的总数量，得到准确率。   
accuracy(x_sample, x) / len(x)

0.5